<h1>Markov NLP</h1>

In [167]:
import pandas as pd
import numpy as np
from datetime import datetime
import networkx as nx
from networkx.convert_matrix import from_pandas_edgelist
from networkx import DiGraph, Graph
from networkx.drawing.nx_pylab import draw_networkx_nodes
import matplotlib.pyplot as plt
from stellargraph import StellarGraph
from pomegranate import BayesianNetwork
import seaborn
import time
import networkx
from pomegranate.utils import plot_networkx
import operator
import re
import collections

<b>Create start timestamp to calculate notebook runtime at bottom</b>

In [168]:
start = datetime.now()
print(start)

2021-07-11 22:16:50.033740


In [169]:
# markov_chain_perc = markov_chain(brand_data, 'rm_dup_brand', 3, True)
# markov_chain_num = markov_chain(brand_data, 'rm_dup_brand', 3, False)

In [170]:
df = pd.read_csv('../../../finance/crypto/reddit/wallstreetbets_comments.csv')

In [171]:
df.shape

(699539, 10)

In [172]:
df.head()

,author,post,created_utc,subreddit,score,permalink,parent_id,id,link_id,is_submitter
0,parker2020,Hate to see it,1617465910,wallstreetbets,1,/r/wallstreetbets/comments/mhs01t/unpinned_dai...,t1_gt97zak,gt98ckt,t3_mhs01t,False
1,FannyPackPhantom,It’s 100% better than any of the girls or high...,1617465909,wallstreetbets,1,/r/wallstreetbets/comments/mhs01t/unpinned_dai...,t1_gt97lzp,gt98ci7,t3_mhs01t,False
2,8an5,"So many triangles, it must be a message from t...",1617465899,wallstreetbets,1,/r/wallstreetbets/comments/mj5rgl/gme_will_we_...,t3_mj5rgl,gt98btc,t3_mj5rgl,False
3,AllRealTruth,Much better than buying a stock at $2 .. selli...,1617465892,wallstreetbets,1,/r/wallstreetbets/comments/mj5rgl/gme_will_we_...,t1_gt8phb6,gt98ban,t3_mj5rgl,False
4,derficker69,Are you aware that you are supporting [this bu...,1617465892,wallstreetbets,1,/r/wallstreetbets/comments/m7bbb9/added_more_p...,t1_gra9hei,gt98bae,t3_m7bbb9,False


In [173]:
df.iloc[45]['post']

'Chuck Norris trades on Saturday, Sunday and on all statutory holidays.'

In [174]:
def clean_and_tokenize(text):
    text = str(text).lower() \
    .replace("it's", "it is") \
    .replace("we'll", "we will") \
    .replace("she'll", "she will") \
    .replace("he'll", "he will") \
    .replace("hasn't", "has not") \
    .replace("isn't", "is not") \
    .replace("won't", "will not") \
    .replace("shouldn't", "should not") \
    .replace(",", "")
    
    text = re.sub('oo+', 'oo', text)
    text = re.sub('aa+', 'a', text)
    text = re.sub('ee+', 'ee', text)
    text = re.sub('ii+', 'i', text)
    text = re.sub('uu+', 'u', text)
    text = re.sub('  ', ' ', text)
    
    rest_of_alphabet = ['b', 'c', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'y', 'z']
    for i in rest_of_alphabet:
        text = re.sub(f'{i}{i}{i}+', f'{i}',text)
    
    return text.split(' ')

df['cleaned_post'] = df['post'].apply(lambda x: clean_and_tokenize(x))
df['cleaned_post_length'] = df['cleaned_post'].apply(lambda x: len(x))
df = df[df['cleaned_post_length'] > 3]

In [175]:
df.iloc[45]['cleaned_post']

['tsm',
 'is',
 'my',
 'favorite',
 'esports',
 'team',
 'there',
 'for',
 'ima',
 'go',
 'for',
 'it']

In [176]:
df.shape

(579047, 12)

In [177]:
df['cleaned_post_length'].sum()

10342707

In [178]:
# def build_vocab(df, sequence_column):
#     vocab = []
#     for i in range(len(df)):
#         if i % 50000 == 0:
#             print(i)
#         for j in df.iloc[i][sequence_column]:
#             if j in vocab:
#                 pass
#             else:
#                 vocab.append(j)
#     print(f'{len(vocab)} word vocabulary created')
#     return vocab

In [179]:
def build_vocab(df, sequence_column):
    all_words = []
    for i in range(len(df)):
        if i % 50000 == 0:
            print(i)
        for j in df.iloc[i][sequence_column]:
                all_words.append(j)
    c = dict(collections.Counter(all_words))
    return [i for i in c.keys()]

In [180]:
vocab = build_vocab(df, 'cleaned_post')

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000


In [181]:
len(vocab)

336740

In [182]:
def markov_chain(df, vocab, sequence_column, k=3):
    print(datetime.now())
    markov_dict = {}

    for i in range(len(df)):
        if i % 50000 == 0:
            print(i)
        listy = df.iloc[i][sequence_column]

        for l, m in enumerate(listy):
            if l < len(listy)-(k+1):
                sequence = (' '.join(listy[l:l+k]))
#                 print(sequence)
                if sequence in markov_dict.keys():
#                     print('a')
                    try:
                        markov_dict[sequence][listy[l+k]] +=1
                    except KeyError:
                        markov_dict[sequence][listy[l+k]] = 1
                else: 
#                     print('b')
                    markov_dict[sequence] = {listy[l+k]:1}
#                     markov_dict[sequence][listy[l+k+1]] += 1
           
#     if percent==True:                
#         for i in markov_dict.keys():
#             total = sum(markov_dict[i].values())
#             for j in vocab:
#                 new_value = (markov_dict[i][j])/total
#                 markov_dict[i][j] = new_value
                
    print(datetime.now())            
    return markov_dict

In [183]:
wsb = markov_chain(df, vocab, 'cleaned_post', k=5)

2021-07-11 22:18:07.749598
0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
2021-07-11 22:19:06.133878


In [184]:
def next(sequence, iterations, k):
    """sequence = str
    """
    listy = []
    text = sequence.split(' ')
    
    for i in range(iterations):
        words = " ".join(text[-k:])
        markov = wsb[words]
        total = sum(wsb[sequence].values())
        nexty = max(markov.items(), key=operator.itemgetter(1))[0]
        num = max(markov.items(), key=operator.itemgetter(1))[1]
        perc = (max(markov.items(), key=operator.itemgetter(1))[1])/total
        listy.append([nexty, perc, num])
        text.append(nexty)
    return listy

In [186]:
next("what are you going to", 10, 5)

[['do', 0.8148148148148148, 22],
 ['with', 0.4444444444444444, 12],
 ['the', 0.1111111111111111, 3],
 ['$1.4m', 0.037037037037037035, 1],
 ['if', 0.037037037037037035, 1],
 ['it', 0.037037037037037035, 1],
 ['is', 0.037037037037037035, 1],
 ['enough', 0.037037037037037035, 1],
 ['to', 0.037037037037037035, 1],
 ['retire', 0.037037037037037035, 1]]

In [188]:
wsb["what are you going to"]

{'do': 22, 'call': 1, 'buy?': 1, 'sell?': 1, 'grow': 1, 'do?': 1}

In [ ]:
end = datetime.now()
print(end)
print(end-start)